In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data = pd.read_csv('result/atepc_result.csv')
data.head(10)

,sentence,IOB,tokens,aspect,position,sentiment,probs,confidence
0,"Brg has arrived , everything is ok , the size ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['Brg', 'has', 'arrived', ',', 'everything', '...",['size'],[[9]],['Positive'],"[[0.0001386514923069626, 0.005269442684948444,...",[0.9946]
1,some items can be found cheaper easy to use too,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-AS...","['some', 'items', 'can', 'be', 'found', 'cheap...",['use'],[[8]],['Positive'],"[[0.00042962649604305625, 0.000955453317146748...",[0.9986]
2,i like shopee and the sellers i have encounter...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['i', 'like', 'shopee', 'and', 'the', 'sellers...",[],[],[],[],[]
3,good good product quality good good product qu...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['good', 'good', 'product', 'quality', 'good',...",[],[],[],[],[]
4,safe until everything is in good condition,"['O', 'O', 'O', 'O', 'O', 'O', 'O']","['safe', 'until', 'everything', 'is', 'in', 'g...",[],[],[],[],[]
5,delivery was okay didnt wrap with extra wraps ...,"['B-ASP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '...","['delivery', 'was', 'okay', 'didnt', 'wrap', '...",['delivery'],[[0]],['Negative'],"[[0.6984570622444153, 0.2885409891605377, 0.01...",[0.6985]
6,good experience tq,"['O', 'B-ASP', 'O']","['good', 'experience', 'tq']",['experience'],[[1]],['Positive'],"[[0.01336316205561161, 0.002318957122042775, 0...",[0.9843]
7,size is very small like xl,"['B-ASP', 'O', 'O', 'O', 'O', 'O']","['size', 'is', 'very', 'small', 'like', 'xl']",['size'],[[0]],['Negative'],"[[0.9988647699356079, 0.0008667182992212474, 0...",[0.9989]
8,wide range of products at reasonable prices,"['O', 'O', 'O', 'O', 'O', 'O', 'B-ASP']","['wide', 'range', 'of', 'products', 'at', 'rea...",['prices'],[[6]],['Positive'],"[[0.002016499638557434, 0.0006462078890763223,...",[0.9973]
9,best service original and fast delivery,"['O', 'B-ASP', 'O', 'O', 'O', 'B-ASP']","['best', 'service', 'original', 'and', 'fast',...","['service', 'delivery']","[[1], [5]]","['Positive', 'Positive']","[[0.004312810488045216, 0.0006725043058395386,...","[0.995, 0.9938]"


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5600 entries, 0 to 5599
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sentence    5600 non-null   object
 1   IOB         5600 non-null   object
 2   tokens      5600 non-null   object
 3   aspect      5600 non-null   object
 4   position    5600 non-null   object
 5   sentiment   5600 non-null   object
 6   probs       5600 non-null   object
 7   confidence  5600 non-null   object
dtypes: object(8)
memory usage: 350.1+ KB


In [4]:
# the only columns we are interested in are apsect_term, sentiment and confidence

# data = data[['aspect', 'sentiment', 'confidence']]

In [5]:
# we drop the ones with no aspects terms
data = data[data['aspect'] != '[]']
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4914 entries, 0 to 5599
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sentence    4914 non-null   object
 1   IOB         4914 non-null   object
 2   tokens      4914 non-null   object
 3   aspect      4914 non-null   object
 4   position    4914 non-null   object
 5   sentiment   4914 non-null   object
 6   probs       4914 non-null   object
 7   confidence  4914 non-null   object
dtypes: object(8)
memory usage: 345.5+ KB


In [6]:
# cleaning the data

data.loc[:, 'aspect'] = data['aspect'].str.replace('[', '')
data.loc[:, 'aspect'] = data['aspect'].str.replace(']', '')
data.loc[:, 'aspect'] = data['aspect'].str.replace("'", '')

data.loc[:, 'sentiment'] = data['sentiment'].str.replace('[', '')
data.loc[:, 'sentiment'] = data['sentiment'].str.replace(']', '')
data.loc[:, 'sentiment'] = data['sentiment'].str.replace("'", '')

data.loc[:, 'confidence'] = data['confidence'].str.replace('[', '')
data.loc[:, 'confidence'] = data['confidence'].str.replace(']', '')
data.loc[:, 'confidence'] = data['confidence'].str.replace("'", '')

aspect_df = data['aspect'].str.split(
    ',', expand=True).stack().reset_index(level=1, drop=True)
sentiment_df = data['sentiment'].str.split(
    ',', expand=True).stack().reset_index(level=1, drop=True).str.strip()
confidence_df = data['confidence'].astype(str).str.split(
    ',', expand=True).stack().reset_index(level=1, drop=True)

# Join the separate dataframes back together to create the final dataframe
final_data = pd.DataFrame({
    'aspect': aspect_df,
    'sentiment': sentiment_df,
    'confidence': confidence_df
})

# assert all columns are of the same length
assert (len(aspect_df) == len(sentiment_df) == len(confidence_df))

# Convert confidence column back to float type
final_data['aspect'] = final_data['aspect'].str.strip()
final_data['sentiment'] = final_data['sentiment'].str.strip()
final_data['confidence'] = final_data['confidence'].astype(float)

data = final_data

data.head(10)

,aspect,sentiment,confidence
0,size,Positive,0.9946
1,use,Positive,0.9986
5,delivery,Negative,0.6985
6,experience,Positive,0.9843
7,size,Negative,0.9989
8,prices,Positive,0.9973
9,service,Positive,0.9950
9,delivery,Positive,0.9938
10,advertisements,Negative,0.9979
11,platform,Positive,0.9985


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7273 entries, 0 to 5599
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   aspect      7273 non-null   object 
 1   sentiment   7273 non-null   object 
 2   confidence  7273 non-null   float64
dtypes: float64(1), object(2)
memory usage: 227.3+ KB


In [8]:
unique_aspects = data['aspect'].unique()
len(unique_aspects)
# data needs to be cleaned

1193

## Visualising Results


In [10]:
import plotly.express as px

aspect_counts = data['aspect'].value_counts()

# Select the top N aspects to display (optional)
N = 20
top_aspects = aspect_counts.head(N)

fig = px.bar(x=top_aspects.index, y=top_aspects.values, labels={
             'x': 'Aspect', 'y': 'Frequency'}, title=f'Top {N} Aspects')
fig.update_layout(autosize=False, width=800, height=500)
# fig.update_xaxes(tickangle=90)
fig.show()

In [12]:
import plotly.express as px
from collections import Counter
import ast

sentiment_freq = Counter(data['sentiment'].values)

# Create a pie chart for Sentiment Distribution
fig = px.pie(values=list(sentiment_freq.values()), names=list(
    sentiment_freq.keys()), title='Sentiment Distribution')
fig.show()

In [13]:
# Count the number of comments for each aspect


# Get the aspects that have 10 or more comments
aspects_with_10_or_more_comments = aspect_counts[aspect_counts >= 10].index

# Filter the data to include only the aspects with 10 or more comments
filtered_data = data[data['aspect'].isin(aspects_with_10_or_more_comments)]

# Calculate the average confidence level for each aspect
average_confidence = filtered_data.groupby(
    'aspect')['confidence'].mean().reset_index()

# Plotly visualization
fig = px.bar(average_confidence, x='aspect', y='confidence', title='Average Aspect Confidence Analysis', labels={
    'confidence': 'Average Confidence Level', 'aspect': 'Aspect'})
fig.show()

In [17]:
# yes

import pandas as pd
import json
import plotly.express as px


# Group by aspect and sentiment and count occurrences
aspect_sentiment_count = data.groupby(
    ['aspect', 'sentiment']).size().reset_index(name='Count')

COUNT_THRESHOLD = 10

# Filter to only include aspects with more than 20 comments
aspect_sentiment_count = aspect_sentiment_count[aspect_sentiment_count['Count'] > COUNT_THRESHOLD]


# Create a bar chart for the count of positive and negative comments for each aspect
fig = px.bar(aspect_sentiment_count, x='aspect', y='Count', color='sentiment', title='Count of Positive and Negative Comments per Aspect',
             labels={'Count': 'Number of Comments', 'Aspect': 'Aspect', 'Sentiment': 'Sentiment'},
             color_discrete_map={'positive': 'green', 'neutral': 'blue', 'negative': 'red'})
fig.update_layout(barmode='group')
fig.show()

In [15]:

fig_confidence_distribution = px.box(data, x='sentiment', y='confidence',
                                     title="Distribution of Confidence Scores by Sentiment",
                                     labels={'sentiment': 'Sentiment', 'confidence': 'Confidence Score'})
fig_confidence_distribution.show()